In [1]:
import fridge
import fridge.utilities.h5Interface as h5i
import fridge.utilities.reactorInterface as RI
import os

from scipy.interpolate import interp1d as interp
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import bb_sfr_opt
import time

# SFR Optimization

This example seeks to optimize a SFR design utilizing the blackboard framework.
We start first by taking a look at the SFR design data using the `h5` add `reactor` interfaces built into `FRDIGe` [cite].
For this we'll examine the objectives that will drive the optimization process: cycle length, plutonium mass, reactivity swing, and core average burnup.
Upon examination, we note that we have removed a significant portion of the design space by ensuring that the reactor design has a cycle length greater than 0 days.

In [3]:
h5_interface = h5i.h5Interface()
h5_interface.read_h5('/Users/ryanstewart/projects/sfr_database/SFR_DB.h5')

In [8]:
cycle_length_list = []
rx_swing_list = []
height = []
smear = []
pu_content = []
bu = []
pu_mass = []
for rx, core in h5_interface.h5file.items():
    try:
        rx_ = RI.reactorInterface(core)
        cycle_length = rx_.extrapolate_value('keff', 'time', 1.0)
        rx_swing = rx_.get_reactivity_swing(1.0, cycle_length)
        rx_swing_per_day = rx_swing/cycle_length * 30
        avg_bu = rx_.get_assembly_avg(cycle_length, 'burnup')
        avg_bu_per_day = avg_bu/cycle_length * 30
        pu = rx_.get_assembly_pu_mass() * 78 / 4
        if cycle_length > 180:
            height.append(rx_.rx['independent variables']['height'][0])
            smear.append(rx_.rx['independent variables']['smear'][0])
            pu_content.append(rx_.rx['independent variables']['pu_content'][0])
            cycle_length_list.append(cycle_length)
            rx_swing_list.append(rx_swing_per_day)
            bu.append(avg_bu_per_day)
            pu_mass.append(pu)
    except (KeyError, TypeError):
        print('didnt get {}'.format(core))

In [9]:
fig = px.scatter_3d(x=height, y=smear, z=cycle_length_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Cycle Length (days)','color':'Pu Fraction'})
fig.show()
fig2 = px.scatter_3d(x=height, y=smear, z=rx_swing_list, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Rx Swing (pcm)','color':'Pu Fraction'})
fig2.show()
fig3 = px.scatter_3d(x=height, y=smear, z=bu, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Burnup (GWd/MTU)','color':'Pu Fraction'})
fig3.show()
fig4 = px.scatter_3d(x=height, y=smear, z=pu_mass, color=pu_content, labels={'x':'Height (cm)', 'y': 'Smear', 'z':'Pu Mass per Cycle (kg)','color':'Pu Fraction'})
fig4.show()

# Starting the Blackboard System

In [2]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_sfr_opt.BbSfrOpt)
bb.set_attr(desired_results={'cycle length': (180, 720), 
                             'reactivity swing': (540, 950), 
                             'burnup': (3.0, 5.5), 
                             'pu mass': (0, 1891)})
bb.generate_sm()

bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_1')
bb.connect_agent(karp.KaRpExplore, 'ka_rp_explore_2')
bb.connect_agent(karp.KaRpExploit, 'ka_rp_exploit')
bb.connect_agent(kabr.KaBr_lvl2, 'ka_br_lvl2')
bb.connect_agent(kabr.KaBr_lvl3, 'ka_br_lvl3')

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:13097 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:13097
INFO [2020-05-06 16:56:42.878231] (bb): Connected agent ka_rp_explore_1 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-06 16:56:42.928931] (bb): Connected agent ka_rp_explore_2 of agent type <class 'ka_rp.KaRpExplore'>
INFO [2020-05-06 16:56:42.981695] (bb): Connected agent ka_rp_exploit of agent type <class 'ka_rp.KaRpExploit'>
INFO [2020-05-06 16:56:43.026850] (bb): Connected agent ka_br_lvl2 of agent type <class 'ka_br.KaBr_lvl2'>
INFO [2020-05-06 16:56:43.077591] (bb): Connected agent ka_br_lvl3 of agent type <class 'ka_br.KaBr_lvl3'>
INFO [2020-05-06 16:56:46.098099] (bb): Selecting agent ka_rp_explore_1 (TV: 0.25) to execute (TE: 1)
INFO [2020-05-06 16:56:49.113942] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 2)
INFO [2020-05-06 16:56:52.128431] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 3)
INFO [2020-05-06

INFO [2020-05-06 16:58:51.085114] (ka_br_lvl3): Core core_[61.47, 62.55, 0.82] Fitness: 1.6526517300039354
INFO [2020-05-06 16:58:52.570033] (bb): Selecting agent ka_rp_explore_1 (TV: 0.75) to execute (TE: 43)
INFO [2020-05-06 16:58:52.587834] (ka_br_lvl3): Core core_[61.47, 62.55, 0.82] Fitness: 1.6526517300039354
INFO [2020-05-06 16:58:55.584047] (bb): Selecting agent ka_rp_explore_2 (TV: 0.75) to execute (TE: 44)
INFO [2020-05-06 16:58:55.602792] (ka_br_lvl3): Core core_[61.47, 62.55, 0.82] Fitness: 1.6526517300039354
INFO [2020-05-06 16:58:58.595306] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 45)
INFO [2020-05-06 16:58:58.612655] (ka_br_lvl3): Core core_[61.47, 62.55, 0.82] Fitness: 1.6526517300039354
INFO [2020-05-06 16:59:01.608190] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 46)
INFO [2020-05-06 16:59:01.628177] (ka_br_lvl3): Core core_[61.47, 62.55, 0.82] Fitness: 1.6526517300039354
INFO [2020-05-06 16:59:04.619530] (bb): Selecting agent

INFO [2020-05-06 17:00:50.015725] (ka_br_lvl3): Core core_[64.37, 68.7, 0.37] Fitness: 2.641051037398727
INFO [2020-05-06 17:00:53.002049] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 83)
INFO [2020-05-06 17:00:54.526329] (ka_br_lvl3): Core core_[56.87, 63.49, 0.68] Fitness: 1.8816347071146389
INFO [2020-05-06 17:00:56.010956] (bb): Selecting agent ka_rp_explore_1 (TV: 0.75) to execute (TE: 84)
INFO [2020-05-06 17:00:56.036732] (ka_br_lvl3): Core core_[56.87, 63.49, 0.68] Fitness: 1.8816347071146389
INFO [2020-05-06 17:00:59.022906] (bb): Selecting agent ka_rp_explore_2 (TV: 0.75) to execute (TE: 85)
INFO [2020-05-06 17:00:59.050740] (ka_br_lvl3): Core core_[56.87, 63.49, 0.68] Fitness: 1.8816347071146389
INFO [2020-05-06 17:01:02.032939] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 86)
INFO [2020-05-06 17:01:02.055538] (ka_br_lvl3): Core core_[56.87, 63.49, 0.68] Fitness: 1.8816347071146389
INFO [2020-05-06 17:01:05.041386] (bb): Selecting agent ka_br_lv

INFO [2020-05-06 17:03:02.472137] (ka_br_lvl3): Core core_[77.24, 61.86, 0.44] Fitness: 2.6313485226897293
INFO [2020-05-06 17:03:05.451651] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 125)
INFO [2020-05-06 17:03:05.482076] (ka_br_lvl3): Core core_[77.24, 61.86, 0.44] Fitness: 2.6313485226897293
INFO [2020-05-06 17:03:08.458784] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 126)
INFO [2020-05-06 17:03:11.471154] (bb): Selecting agent ka_rp_explore_2 (TV: 0.75) to execute (TE: 127)
INFO [2020-05-06 17:03:12.032444] (ka_br_lvl3): Core core_[73.25, 65.03, 0.15] Fitness: 3.359666019944928
INFO [2020-05-06 17:03:12.046920] (ka_br_lvl3): Core core_[73.25, 65.03, 0.15] Fitness: 3.359666019944928
INFO [2020-05-06 17:03:14.480079] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 128)
INFO [2020-05-06 17:03:17.033743] (ka_br_lvl3): Core core_[77.68, 61.94, 0.61] Fitness: 2.219197741731478
INFO [2020-05-06 17:03:17.492599] (bb): Selecting agent ka_rp_explore

INFO [2020-05-06 17:05:11.911585] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 163)
INFO [2020-05-06 17:05:14.922691] (bb): Selecting agent ka_rp_explore_1 (TV: 1.0) to execute (TE: 164)
INFO [2020-05-06 17:05:17.931642] (bb): Selecting agent ka_rp_explore_2 (TV: 1.0) to execute (TE: 165)
INFO [2020-05-06 17:05:19.521571] (ka_br_lvl3): Core core_[58.707, 65.82, 0.66] Fitness: 1.9319260760968815
INFO [2020-05-06 17:05:19.541721] (ka_br_lvl3): Core core_[58.707, 65.82, 0.66] Fitness: 1.9319260760968815
INFO [2020-05-06 17:05:19.560322] (ka_br_lvl3): Core core_[58.707, 65.82, 0.66] Fitness: 1.9319260760968815
INFO [2020-05-06 17:05:20.940072] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 166)
INFO [2020-05-06 17:05:20.981694] (ka_br_lvl3): Core core_[58.707, 65.82, 0.66] Fitness: 1.9319260760968815
INFO [2020-05-06 17:05:23.948070] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 167)
INFO [2020-05-06 17:05:23.989653] (ka_br_lvl3): Core core_[5

...

ERROR [2020-05-06 18:18:52.992829] (ka_rp_exploit): An exception occurred while running! (dictionary changed size during iteration)

|>>>>>>>>Traceback (most recent call last):
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1680, in run
|     self._loop()
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1208, in _loop
|     if self._iterate():
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1245, in _iterate
|     self._process_events(events)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1282, in _process_events
|     self._process_single_event(socket)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/site-packages/osbrain/agent.py", line 1296, in _process_single_event
|     self._process_sub_event(socket, address, data)
|   File "/Users/ryanstewart/Library/Python/3.7/lib/python/sit

INFO [2020-05-06 18:18:54.152385] (ka_br_lvl3): Core core_[63.05, 62.47, 0.64] Fitness: 2.0535807340526446
INFO [2020-05-06 18:18:54.278932] (ka_br_lvl3): Core core_[63.05, 62.47, 0.64] Fitness: 2.0535807340526446
INFO [2020-05-06 18:18:54.404888] (ka_br_lvl3): Core core_[63.05, 62.47, 0.64] Fitness: 2.0535807340526446
INFO [2020-05-06 18:18:55.456217] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 1573)
INFO [2020-05-06 18:18:58.466097] (bb): Selecting agent ka_rp_explore_1 (TV: 0.75) to execute (TE: 1574)
INFO [2020-05-06 18:19:01.476341] (bb): Selecting agent ka_rp_explore_2 (TV: 0.75) to execute (TE: 1575)
INFO [2020-05-06 18:19:04.486357] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1576)
INFO [2020-05-06 18:19:07.495483] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1577)
INFO [2020-05-06 18:19:07.644435] (ka_br_lvl3): Core core_[70.14, 68.06, 0.24] Fitness: 2.992436812116569
INFO [2020-05-06 18:19:10.508883] (bb): Selecting agent ka

INFO [2020-05-06 18:20:55.925311] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1613)
INFO [2020-05-06 18:20:56.079631] (ka_br_lvl3): Core core_[73.64, 59.7, 0.83] Fitness: 1.7730553166723442
INFO [2020-05-06 18:20:58.938744] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1614)
INFO [2020-05-06 18:20:59.092856] (ka_br_lvl3): Core core_[73.64, 59.7, 0.83] Fitness: 1.7730553166723442
INFO [2020-05-06 18:21:01.953676] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1615)
INFO [2020-05-06 18:21:02.105715] (ka_br_lvl3): Core core_[73.64, 59.7, 0.83] Fitness: 1.7730553166723442
INFO [2020-05-06 18:21:04.964990] (bb): Selecting agent ka_br_lvl3 (TV: 3) to execute (TE: 1616)
INFO [2020-05-06 18:21:07.979452] (bb): Selecting agent ka_rp_explore_2 (TV: 0.75) to execute (TE: 1617)
INFO [2020-05-06 18:21:08.940422] (ka_br_lvl3): Core core_[69.11, 56.48, 0.57] Fitness: 2.2786817384722395
INFO [2020-05-06 18:21:09.076347] (ka_br_lvl3): Core core_[69

INFO [2020-05-06 18:22:53.589056] (ka_br_lvl3): Core core_[68.51, 67.29, 0.37] Fitness: 2.667762496499114
INFO [2020-05-06 18:22:56.393810] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1653)
INFO [2020-05-06 18:22:56.552471] (ka_br_lvl3): Core core_[68.51, 67.29, 0.37] Fitness: 2.667762496499114
INFO [2020-05-06 18:22:59.404844] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1654)
INFO [2020-05-06 18:22:59.562803] (ka_br_lvl3): Core core_[68.51, 67.29, 0.37] Fitness: 2.667762496499114
INFO [2020-05-06 18:23:02.419575] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1655)
INFO [2020-05-06 18:23:02.576591] (ka_br_lvl3): Core core_[68.51, 67.29, 0.37] Fitness: 2.667762496499114
INFO [2020-05-06 18:23:05.430328] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1656)
INFO [2020-05-06 18:23:05.584420] (ka_br_lvl3): Core core_[68.51, 67.29, 0.37] Fitness: 2.667762496499114
INFO [2020-05-06 18:23:08.443346] (bb): Selecting agen

INFO [2020-05-06 19:02:27.358448] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1692)
INFO [2020-05-06 19:02:27.515888] (ka_br_lvl3): Core core_[65.04, 63.38, 0.68] Fitness: 2.0048915607416964
INFO [2020-05-06 19:02:30.366229] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1693)
INFO [2020-05-06 19:02:30.520863] (ka_br_lvl3): Core core_[65.04, 63.38, 0.68] Fitness: 2.0048915607416964
INFO [2020-05-06 19:02:33.376198] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1694)
INFO [2020-05-06 19:02:33.544317] (ka_br_lvl3): Core core_[65.04, 63.38, 0.68] Fitness: 2.0048915607416964
INFO [2020-05-06 19:02:36.389442] (bb): Selecting agent ka_rp_explore_2 (TV: 0.5) to execute (TE: 1695)
INFO [2020-05-06 19:02:36.545077] (ka_br_lvl3): Core core_[65.04, 63.38, 0.68] Fitness: 2.0048915607416964
INFO [2020-05-06 19:02:39.401374] (bb): Selecting agent ka_rp_explore_1 (TV: 0.5) to execute (TE: 1696)
INFO [2020-05-06 19:02:39.612329] (ka_br_lvl3): Core

In [3]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    time.sleep(3)
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    bb.determine_complete()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))    

Finished trigger event: 1
Finished trigger event: 2
Finished trigger event: 3
Finished trigger event: 4
Finished trigger event: 5
Finished trigger event: 6
Finished trigger event: 7
Finished trigger event: 8
Finished trigger event: 9
Finished trigger event: 10
Finished trigger event: 11
Finished trigger event: 12
Finished trigger event: 13
Finished trigger event: 14
Finished trigger event: 15
Finished trigger event: 16
Finished trigger event: 17
Finished trigger event: 18
Finished trigger event: 19
Finished trigger event: 20
Finished trigger event: 21
Finished trigger event: 22
Finished trigger event: 23
Finished trigger event: 24
Finished trigger event: 25
Finished trigger event: 26
Finished trigger event: 27
Finished trigger event: 28
Finished trigger event: 29
Finished trigger event: 30
Finished trigger event: 31
Finished trigger event: 32
Finished trigger event: 33
Finished trigger event: 34
Finished trigger event: 35
Finished trigger event: 36
Finished trigger event: 37
Finished t

Finished trigger event: 298
Finished trigger event: 299
Finished trigger event: 300
Finished trigger event: 301
Finished trigger event: 302
Finished trigger event: 303
Finished trigger event: 304
Finished trigger event: 305
Finished trigger event: 306
Finished trigger event: 307
Finished trigger event: 308
Finished trigger event: 309
Finished trigger event: 310
Finished trigger event: 311
Finished trigger event: 312
Finished trigger event: 313
Finished trigger event: 314
Finished trigger event: 315
Finished trigger event: 316
Finished trigger event: 317
Finished trigger event: 318
Finished trigger event: 319
Finished trigger event: 320
Finished trigger event: 321
Finished trigger event: 322
Finished trigger event: 323
Finished trigger event: 324
Finished trigger event: 325
Finished trigger event: 326
Finished trigger event: 327
Finished trigger event: 328
Finished trigger event: 329
Finished trigger event: 330
Finished trigger event: 331
Finished trigger event: 332
Finished trigger eve

Finished trigger event: 591
Finished trigger event: 592
Finished trigger event: 593
Finished trigger event: 594
Finished trigger event: 595
Finished trigger event: 596
Finished trigger event: 597
Finished trigger event: 598
Finished trigger event: 599
Finished trigger event: 600
Finished trigger event: 601
Finished trigger event: 602
Finished trigger event: 603
Finished trigger event: 604
Finished trigger event: 605
Finished trigger event: 606
Finished trigger event: 607
Finished trigger event: 608
Finished trigger event: 609
Finished trigger event: 610
Finished trigger event: 611
Finished trigger event: 612
Finished trigger event: 613
Finished trigger event: 614
Finished trigger event: 615
Finished trigger event: 616
Finished trigger event: 617
Finished trigger event: 618
Finished trigger event: 619
Finished trigger event: 620
Finished trigger event: 621
Finished trigger event: 622
Finished trigger event: 623
Finished trigger event: 624
Finished trigger event: 625
Finished trigger eve

Finished trigger event: 884
Finished trigger event: 885
Finished trigger event: 886
Finished trigger event: 887
Finished trigger event: 888
Finished trigger event: 889
Finished trigger event: 890
Finished trigger event: 891
Finished trigger event: 892
Finished trigger event: 893
Finished trigger event: 894
Finished trigger event: 895
Finished trigger event: 896
Finished trigger event: 897
Finished trigger event: 898
Finished trigger event: 899
Finished trigger event: 900
Finished trigger event: 901
Finished trigger event: 902
Finished trigger event: 903
Finished trigger event: 904
Finished trigger event: 905
Finished trigger event: 906
Finished trigger event: 907
Finished trigger event: 908
Finished trigger event: 909
Finished trigger event: 910
Finished trigger event: 911
Finished trigger event: 912
Finished trigger event: 913
Finished trigger event: 914
Finished trigger event: 915
Finished trigger event: 916
Finished trigger event: 917
Finished trigger event: 918
Finished trigger eve

Finished trigger event: 1171
Finished trigger event: 1172
Finished trigger event: 1173
Finished trigger event: 1174
Finished trigger event: 1175
Finished trigger event: 1176
Finished trigger event: 1177
Finished trigger event: 1178
Finished trigger event: 1179
Finished trigger event: 1180
Finished trigger event: 1181
Finished trigger event: 1182
Finished trigger event: 1183
Finished trigger event: 1184
Finished trigger event: 1185
Finished trigger event: 1186
Finished trigger event: 1187
Finished trigger event: 1188
Finished trigger event: 1189
Finished trigger event: 1190
Finished trigger event: 1191
Finished trigger event: 1192
Finished trigger event: 1193
Finished trigger event: 1194
Finished trigger event: 1195
Finished trigger event: 1196
Finished trigger event: 1197
Finished trigger event: 1198
Finished trigger event: 1199
Finished trigger event: 1200
Finished trigger event: 1201
Finished trigger event: 1202
Finished trigger event: 1203
Finished trigger event: 1204
Finished trigg

Finished trigger event: 1454
Finished trigger event: 1455
Finished trigger event: 1456
Finished trigger event: 1457
Finished trigger event: 1458
Finished trigger event: 1459
Finished trigger event: 1460
Finished trigger event: 1461
Finished trigger event: 1462
Finished trigger event: 1463
Finished trigger event: 1464
Finished trigger event: 1465
Finished trigger event: 1466
Finished trigger event: 1467
Finished trigger event: 1468
Finished trigger event: 1469
Finished trigger event: 1470
Finished trigger event: 1471
Finished trigger event: 1472
Finished trigger event: 1473
Finished trigger event: 1474
Finished trigger event: 1475
Finished trigger event: 1476
Finished trigger event: 1477
Finished trigger event: 1478
Finished trigger event: 1479
Finished trigger event: 1480
Finished trigger event: 1481
Finished trigger event: 1482
Finished trigger event: 1483
Finished trigger event: 1484
Finished trigger event: 1485
Finished trigger event: 1486
Finished trigger event: 1487
Finished trigg

KeyboardInterrupt: 